In [ ]:
import glob

# check our work directory
import os

# to unzip datasets
import zipfile

# visualize some datasets
import matplotlib.pyplot as plt

# using numpy
import numpy as np

# for data load or save
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms

plt.style.use(["dark_background", "bmh"])
plt.rc("axes", facecolor="k")
plt.rc("figure", facecolor="k")
plt.rc("figure", figsize=(10, 10), dpi=100)

In [ ]:
os.listdir("..")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1234)
if device == "cuda":
    torch.cuda.manual_seed_all(1234)
device

In [ ]:
base_dir = "../data/dogs-vs-cats-redux-kernels-edition"
with zipfile.ZipFile(os.path.join(base_dir, "train.zip")) as train_zip:
    train_zip.extractall("../data/dogs-vs-cats-redux-kernels-edition")
with zipfile.ZipFile(os.path.join(base_dir, "test.zip")) as test_zip:
    test_zip.extractall("../data/dogs-vs-cats-redux-kernels-edition")

In [ ]:
train_dir = "../../data/dogs-vs-cats-redux-kernels-edition/train"
test_dir = "../../data/dogs-vs-cats-redux-kernels-edition/test"

train_list = glob.glob(os.path.join(train_dir, "*.jpg"))
test_list = glob.glob(os.path.join(test_dir, "*.jpg"))

print(f"Training samples: {len(train_list)}, Tests samples: {len(test_list)}")

In [ ]:
from PIL import Image

random_idx = np.random.randint(1, 25000, size=10)

fig = plt.figure(figsize=(16, 6))
i = 1
for idx in random_idx:
    ax = fig.add_subplot(2, 5, i)
    img = Image.open(train_list[idx])
    plt.imshow(img)
    i += 1

plt.axis("off")
plt.show()

In [ ]:
print(
    f"Instances dog: {sum('dog.' in s for s in train_list)}, Instances Cat: {sum('cat.' in s for s in train_list)}"
)

In [ ]:
class DogVsCatDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.file_list[idx]
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        label_category = label = img_name.split("/")[-1].split(".")[0]
        if label_category == "dog":
            label = 1
        elif label_category == "cat":
            label = 0

        return image, label

In [ ]:
# Explore more transformations
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)
test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
    ]
)

In [ ]:
train_data = DogVsCatDataset(train_list, transform=train_transforms)
test_data = DogVsCatDataset(test_list, transform=test_transforms)

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    dataset=train_data, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(dataset=test_data, shuffle=True)

In [ ]:
print(len(train_data), len(train_loader))

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.fc1 = nn.Linear(3 * 3 * 64, 10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10, 2)
        self.relu = nn.ReLU()

    # Helps
    # https://datascience.stackexchange.com/questions/40906/determining-size-of-fc-layer-after-conv-layer-in-pytorch
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = CNN().to(device)
model.train()

In [ ]:
# TIPS FOR FINE TUNNING
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

optimizer = optim.Adam(params=model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 1
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for batch_idx, (data, target) in enumerate(train_loader):

        # get data
        data, target = data.to(device), target.to(device)

        # zero grad
        optimizer.zero_grad()

        # predictions
        output = model(data)

        # comput loss and backward
        loss = criterion(output, target)
        loss.backward()

        # adjust weights
        optimizer.step()

        acc = (output.argmax(dim=1) == target).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )
    print(
        f"Epoch : {epoch+1}, train accuracy : {epoch_accuracy}, train loss : {epoch_loss}"
    )

In [ ]:
print(
    f"Epoch : {epoch+1}, train accuracy : {epoch_accuracy}, train loss : {epoch_loss}"
)

In [ ]:
probs = []
model.eval()
with torch.no_grad():
    for data, fileid in test_loader:
        data = data.to(device)
        preds = model(data)
        preds_list = F.softmax(preds, dim=1)[:, 1].tolist()
        probs += list(zip(list(fileid), preds_list))

In [ ]:
from PIL import Image

random_idx = np.random.randint(1, len(test_loader), size=10)

class_ = {0: "cat", 1: "dog"}


fig = plt.figure(figsize=(16, 6))
i = 1

for idx in random_idx:
    ax = fig.add_subplot(2, 5, i)

    img_id = probs[idx][0]
    pred = probs[idx][1]

    if pred > 0.5:
        label = 1
    else:
        label = 0

    img_path = os.path.join(test_dir, f"{img_id}.jpg")
    img = Image.open(img_path)
    plt.imshow(img)
    plt.title(class_[label])

    i += 1

plt.axis("off")
plt.show()

#### What to do?

- Change the code in order to improve the performance. Tips: add more transformation, increase the number of epochs, you can even try to change the model. Maybe you can also use a validation test, by spliting the train_list, and check the validation performance on the end of each epoch
- Try to fine tune a model, the tips are below. 

In [ ]:
# TIPS FOR FINE TUNNING
#  model_conv = models.resnet18(pretrained=True)
# for param in model_conv.parameters():
#     param.requires_grad = False

# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 2)

# model_conv = model_conv.to(device)


# model_conv.fc.parameters() #ADD THIS IN THE OPTIMIZER